# Clase 37

Para una mejor visualización entrar al siguiente [link](https://nbviewer.jupyter.org/github/racsosabe/Miscelanea/blob/master/UPC/Clase%2037%20-%20Estructuras%20de%20Datos%20V.ipynb)

# Requisitos Previos

* Matemática Básica
* Matemática Discreta
* Segment Tree

# Segment Tree + Lazy Propagation

Hasta ahora hemos visto que la estructura del Segment Tree permite responder consultas y realizar modificaciones puntuales en $O(\log{n})$ ejecuciones de la combinación de respuestas parciales (*merge*).

Si tomamos en cuenta lo anterior, si deseamos modificar un rango $[l, r]$, dicha operación nos tomaría $O(n\log{n})$ ejecuciones del *merge*, por lo que no tenemos una forma tan eficiente de realizar modificaciones en rango.

Sin embargo, existe el concepto de *Lazy propagation*, el cual básicamente se puede expresar como: "Si sabes cómo afecta la modificación una respuesta parcial, entonces solo basta con postergar las modificaciones puntuales lo más que se pueda". En términos simples, lo que se propone es aprovechar la propiedad de que todo rango $[l, r]$ está cubierto por $O(\log{n})$ nodos del segment tree, así que solamente debemos saber cómo reflejar esos cambios sobre esos nodos y mantener dichas modificaciones en una "lista de espera".

De esta manera, podemos simplemente aplicar los cambios y mantener información extra por cada nodo que nos señale si este tiene modificaciones pendientes o no (también mantener dichas modificaciones).

## Actualización 

Una característica de las modificaciones es que deben poder acumularse de manera eficiente (apilarse y volverse una sola), para no tener una cola de espera, sino una sola modificación que refleje todas las pendientes. Se puede considerar que si la función de modificación es $f$, tendremos que usar una función compuesta $f_{1}of_{2}o\ldots$ para esto, inicializándola con la identidad.

Deberemos considerar lo siguiente:

1. Para poder actualizar cada nodo que cubre el intervalo $[l, r]$ deberemos cambiar nuestra actualización puntual y ya no descartar ninguno de los dos hijos a menos que este no se intersecte con $[l, r]$.

2. Cada vez que llegamos a un determinado nodo que cubre el intervalo $[l, r]$ debemos actualizar sus modificaciones pendientes con la que se está realizando y detener la recursión en ese momento.

3. Cuando llegamos a un nodo cualquiera, debemos tener su versión actualizada, por lo que se deben efectuar las modificaciones pendientes y pasarlas a sus hijos (de existir).

Tendremos 3 funciones principales:

1. $f(x, y)$: Función para mezclar las respuestas parciales del Segment Tree.

2. $g(x, y)$: Función para efectuar la modificación $y$ sobre la respuesta parcial $x$.

3. $merge(x, y)$: Función para combinar las modificaciones $x$ y $y$ y dar como respuesta una sola modificación.

Entonces, definimos nuestras dos funciones `push(pos, l, r)`, para efectuar las modificaciones pendientes del nodo $pos$, y `update(x, y, z, pos, l, r)`, para actualizar el rango $[x, y]$ con el elemento $z$ (el cual puede ser de cualquier tipo ya que plantearemos una forma general):

```Python
push(pos, l, r):
    st[pos] = g(st[pos], lazy[pos]) # Función para efectuar modificaciones pendientes
    if l < r:
        lazy[2 * pos] = merge(lazy[pos], lazy[2 * pos]) # Función para combinar actualizaciones
        lazy[2 * pos + 1] = merge(lazy[pos], lazy[2 * pos + 1])
    lazy[pos] = NIL
    
    
update(x, y, z, pos, l, r):
    push(pos, l, r)
    if y < l or r < x: return # No está en el intervalo
    if x <= l and r <= y:
        lazy[pos] = z # Podemos asignar de manera directa porque lazy[pos] es NIL en ese momento
        push(pos, l, r)
        return
    mi = (l + r) / 2
    update(x, y, z, 2 * pos, l, mi)
    update(x, y, z, 2 * pos + 1, mi + 1 , r)
    st[pos] = f(st[2 * pos], st[2 * pos + 1]) # Función original para respuestas parciales
```

La complejidad de esta operación es de $O(\log{n}\cdot (f + g + merge))$. Donde $f$, $g$ y $merge$ son las complejidades de evaluar sus respectivas funciones.

## Consulta

Para realizar una consulta, nos basta con efectuar todas las modificaciones de cada nodo antes de continuar con el resto de la función `query(x, y, pos, l, r)`, esta casi no cambia respecto a la original para Segment Tree Simple:

```Python
query(x, y, pos, l, r):
    push(pos, l, r)
    if y < l or r < x: return NIL
    if x <= l and r <= y: return st[pos]
    mi = (l + r) / 2
    return f(query(x, y, 2 * pos, l, mi), query(x, y, 2 * pos + 1, mi + 1, r))
```

Es evidente que la complejidad será de $O(\log{n}\cdot (f + g + merge))$. Donde $f$, $g$ y $merge$ son las complejidades de evaluar sus respectivas funciones.

## Suma en rango

Consideremos el ejemplo más simple de todos, el problema de consultar la suma de elementos en un rango y actualizar los elementos en un rango.

### Modificaciones como función afín

Una función afín está definida como toda función $h$ tal que:

$$ h(x) = ax + b $$

Para algún $a$ y $b$.

Si consideramos que nuestras modificaciones serán aplicar una función afin a cada elemento de un rango, entonces podremos ejecutar 3 posibles modificaciones con la misma estructura:

1) Asignar todos los elementos del rango a $x$: Tomamos $a = 0$, $b = x$

2) Sumar $x$ a todos los elementos del rango: Tomamos $a = 1$, $b = x$.

3) Multiplicar por $x$ a todos los elementos del rango: Tomamos $a = x$, $b = 0$

Esto quiere decir que nuestra estructura $lazy$ va a almacenar pares de números $(a, b)$ y la definición de las funciones $f$ (suma), $g$ (aplicación de h) y $merge$ (composición de $h$) serían las siguientes, considerando que las funciones son respecto a una variable $X$:

$$ f(x, y) = x + y $$
$$ g(x, y) = y_{a}x + y_{b} $$

Estas dos funciones son simples porque su resultado debe ser del mismo tipo que las respuestas parciales; sin embargo, en el caso de la función $merge$ se debe dar como resultado una función afín en forma de par $(a, b)$:

$$ merge(x, y) = x_{a}(y_{a}X + y_{b}) + x_{b} = x_{a}y_{a}X + (x_{a}y_{b} + x_{b}) \rightarrow (x_{a}y_{a}, x_{a}y_{b} + x_{b}) $$

De esta manera obtenemos un Segment Tree bastante flexible y eficiente. 

**Nota:** Es importante recalcar que la principal dificultad del modelamiento de la estructura son las funciones $g$ y $merge$, ya que $g$ es analizar cómo afecta una modificación elemento a elemento al rango completo y la de $merge$ es cómo combinar de manera eficiente y correcta las actualizaciones consecutivas.

## Problemas para practicar

- [CF Edu](https://codeforces.com/edu/course/2/lesson/5)

# Segment Tree Persistente

La persistencia es una técnica que aprovecha la naturaleza de una estructura de datos para poder crear diferentes *versiones* de la misma, dando diferentes posibilidades de modificación. En el caso del Segment Tree, sus aplicaciones más comunes son las de persistencia parcial y completa. Su filosofía es como la de una línea temporal, así que todo lo que es pasado no se puede modificar, pero sí se pueden generar líneas alternas a partir de ese punto.

Analizaremos la estructura de un segment tree antes y después de modificarlo, sean $S$ y $S'$ las dos versiones correspondientes, entonces:

- $S$ y $S'$ solo difieren en $O(\log{n})$ nodos.

- Podemos "reciclar" los nodos de $S$ al momento de realizar la modificación para que $S'$ solamente genere cada nodo que es modificado, logrando una memoria extra de $O(\log{n})$.

![](https://media.geeksforgeeks.org/wp-content/uploads/16709563_1296205210463946_402785728_o.png)

En la imagen anterior se puede ver de color amarillo/anaranjado los nodos nuevos creados por la modificación.

La implementación se puede realizar con punteros, pero tomaremos como referencia al Segment Tree Dinámico para extenderlo con persistencia.

## Isomorfismo

Un punto clave de las modificaciones y los nodos nuevos es el isomorfismo de la versión antigua y la nueva: Para poder reciclar la mayor cantidad de nodos posible, siempre deberemos llevar en nuestra actualización los dos nodos, el que representa al rango $[l, r]$ en la versión anterior y el que lo representa en la nueva.

De esta forma, si el nodo de la versión anterior es $last$ y el de la nueva es $pos$, nuestra función de modificación se volverá:

```Python
update(x, y, last, pos, l, r):
    if l == r:
        st[pos] = y
        return
    mi = l + (r - l) / 2
    if l <= x <= mi:
        R[pos] = R[last] # El hijo derecho no se modifica, así que mantenemos los valores
        L[pos] = addNode(L[last]) # addNode() nos devuelve la posición de un nuevo nodo creado en base a L[last]
        update(x, y, L[last], L[pos], l, mi)
    else:
        L[pos] = L[last] # El hijo izquierdo no se modifica, así que mantenemos los valores
        R[pos] = addNode(R[last]) # addNode() nos devuelve la posición de un nuevo nodo creado en base a L[last]
        update(x, y, R[last], R[pos], mi + 1, r)
    st[pos] = merge(st[L[pos]], st[R[pos]])
```

Ahora realizamos modificaciones en $O(\log{n})$ creando $O(\log{n})$ nodos nuevos.

Por otro lado, la estructura de la consulta depende completamente de la raiz sobre la que se aplique, así que no va a ser modificada; lo único que cambiará será que no la llamaremos con $pos = 0$ siempre, sino que dependerá de la versión que querramos consultar.

```Python
query(x, y, pos, l, r):
    if y < l or r < x: return NIL 
    if x <= l and r <= y: return st[pos]
    mi = l + (r - l) / 2
    return merge(query(x, y, L[pos], l, mi), query(x, y, R[pos], mi + 1, r))
```

## Regresando un tiempo arbitrario

Para poder aplicar cambios a cualquiera de las versiones anteriores, nos basta mantener un vector que nos dé el nodo que representa al rango $[xmin, xmax]$ de cada una de las versiones, así podemos simplemente llamar a la función de modificación y crear la nueva versión. Obviamente tendremos que agregar esta nueva raiz al vector de raíces.

Una implementación de un Segment Tree completamente persistente (no dinámico) con actualización puntual de suma y consulta de suma en rango de alguna versión es la siguiente:

```C++
struct PSegTree{
	int xmin;
	int xmax;
	int nodes;
	vector<int> st;
	vector<int> root;
	vector<int> L, R;
	
	PSegTree(int xmin, int xmax) : xmin(xmin), xmax(xmax) {
		nodes = 0;
		addNode();
		root.emplace_back(0);
		build(0, xmin, xmax);
	}

	void addNode(int x = 0){
		st.emplace_back(x);
		L.emplace_back(-1);
		R.emplace_back(-1);
		nodes += 1;
	}

	void build(int pos, int l, int r){
		if(l == r){
			st[pos] = 0;
			return;
		}
		int mi = l + (r - l) / 2;
		if(L[pos] == -1){
			L[pos] = nodes;
			addNode();
		}
		if(R[pos] == -1){
			R[pos] = nodes;
			addNode();
		}
		st[pos] = st[L[pos]] + st[R[pos]];
	}

	void update(int x, int y, int last, int pos, int l, int r){
		if(l == r){
			st[pos] += y;
			return;
		}
		int mi = l + (r - l) / 2;
		if(l <= x and x <= mi){
			R[pos] = R[last];
			L[pos] = nodes;
			addNode(st[L[last]]);
			update(x, y, L[last], L[pos], l, mi);
		}
		else{
			L[pos] = L[last];
			R[pos] = nodes;
			addNode(st[R[last]]);
			update(x, y, R[last], R[pos], mi + 1, r);
		}
		st[pos] = st[L[pos]] + st[R[pos]];
	}

	int query(int x, int y, int pos, int l, int r){
		if(y < l or r < x or x > y) return 0;
		if(x <= l and r <= y) return st[pos];
		int mi = l + (r - l) / 2;
		return query(x, y, L[pos], l, mi) + query(x, y, R[pos], mi + 1, r);
	}

	void update(int v, int x, int y){ // Modificar la version v
		int last = root[v];
		root.emplace_back(nodes);
		addNode();
		update(x, y, last, nodes - 1, xmin, xmax);
	}

	int query(int v, int x){
		return query(xmin, x, root[v], xmin, xmax);
	}
};
```

### Problema para resolver en clase

- [K-Query Online](https://www.spoj.com/problems/KQUERYO/)